In [ ]:
# import os
# import fitz  # PyMuPDF
# import docx
# import re
# import json
# import requests
# import time
# import spacy
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # ------------------- CONFIG -------------------
# ey_template_path = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\EY_sample_resume_template.txt"
# output_dir = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\final_customised_resumes"
# jd_folder = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\JDs"
# resume_folder = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\Rawresumes"
# os.makedirs(output_dir, exist_ok=True)
# nlp = spacy.load("en_core_web_sm")

# # ------------------- LLM CALL -------------------
# def call_lmstudio(prompt, model="zephyr-7b-beta.Q4_K_M.gguf", retries=3):
#     url = "http://localhost:1234/v1/chat/completions"
#     headers = {"Content-Type": "application/json"}
#     payload = {
#         "model": model,
#         "messages": [{"role": "user", "content": prompt}],
#         "temperature": 0.7,
#         "stream": False
#     }
#     for attempt in range(retries):
#         try:
#             response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=180)
#             return response.json()['choices'][0]['message']['content']
#         except Exception as e:
#             if attempt == retries - 1:
#                 print("LLM call failed:", e)
#                 return ""
#             time.sleep(2)

# # ------------------- UTILITIES -------------------
# def read_text_file(path):
#     with open(path, "r", encoding="utf-8") as f:
#         return f.read()

# def read_resume_text(file_path):
#     if file_path.endswith(".pdf"):
#         doc = fitz.open(file_path)
#         return "\n".join([page.get_text() for page in doc])
#     elif file_path.endswith(".docx"):
#         doc = docx.Document(file_path)
#         return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
#     elif file_path.endswith(".txt"):
#         return read_text_file(file_path)
#     return ""

# def get_best_match(input_text, folder, top_k=1):
#     files = [f for f in os.listdir(folder) if f.lower().endswith((".txt", ".pdf", ".docx"))]
#     contents = []
#     paths = []
#     for f in files:
#         full_path = os.path.join(folder, f)
#         try:
#             contents.append(read_resume_text(full_path))
#             paths.append(full_path)
#         except:
#             continue
#     if not contents:
#         return None, ""
#     tfidf = TfidfVectorizer().fit_transform([input_text] + contents)
#     sims = cosine_similarity(tfidf[0:1], tfidf[1:]).flatten()
#     best_idx = sims.argmax()
#     return paths[best_idx], contents[best_idx]

# # ------------------- ACCURACY METRICS -------------------
# def compute_tfidf_similarity(jd, resume):
#     tfidf = TfidfVectorizer().fit_transform([jd, resume])
#     return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

# def keyword_coverage(jd, resume):
#     jd_keywords = set(token.text.lower() for token in nlp(jd) if token.is_alpha and not token.is_stop)
#     resume_words = resume.lower()
#     matched = [kw for kw in jd_keywords if kw in resume_words]
#     return len(matched) / len(jd_keywords) if jd_keywords else 0, matched

# def qualitative_relevance(jd, resume):
#     jd_words = set(jd.lower().split())
#     resume_words = set(resume.lower().split())
#     overlap = jd_words.intersection(resume_words)
#     return len(overlap) / len(jd_words) if jd_words else 0

# def section_audit(text):
#     sections = ["summary", "qualification", "experience", "technical", "skill"]
#     present = [sec for sec in sections if sec in text.lower()]
#     return present

# # ------------------- MAIN FUNCTION -------------------
# def customise_resume(prompt_text, resume_path):
#     candidate_name = os.path.splitext(os.path.basename(resume_path))[0]

#     print(f"\nStep 1: Retrieving closest-matching JD from local JD folder for → {prompt_text}")
#     jd_path, enriched_jd = get_best_match(prompt_text, jd_folder)
#     if not enriched_jd:
#         print("❌ JD retrieval failed.")
#         return
#     print("✅ JD Retrieved From:", jd_path)

#     print(f"\nStep 2: Reading uploaded resume → {resume_path}")
#     resume_text = read_resume_text(resume_path)

#     print("\nStep 3: If resume sparse, enrich from closest historical resume")
#     if len(resume_text.strip().split()) < 150:
#         _, enriched_resume = get_best_match(prompt_text, resume_folder)
#         resume_text += "\n\n[ENRICHED CONTEXT]\n" + enriched_resume

#     print("\nStep 4: Sending both JD + resume to LLM for customization")
#     custom_prompt = f"""
# You are a resume writer. Given the candidate's resume and the job description, rewrite the following sections to align with the JD. DO NOT invent any fake information.

# Resume:
# {resume_text}

# Job Description:
# {enriched_jd}

# Provide the output in this format (with labels):
# SUMMARY:
# ...

# QUALIFICATIONS:
# ...

# EXPERIENCE:
# ...

# TECHNICAL_SKILLS:
# ...
# """
#     result = call_lmstudio(custom_prompt)
#     if not result:
#         print("❌ Customization failed.")
#         return

#     # Extract sections
#     summary = re.search(r"SUMMARY:(.*?)QUALIFICATIONS:", result, re.DOTALL | re.IGNORECASE)
#     qualifications = re.search(r"QUALIFICATIONS:(.*?)EXPERIENCE:", result, re.DOTALL | re.IGNORECASE)
#     experience = re.search(r"EXPERIENCE:(.*?)TECHNICAL_SKILLS:", result, re.DOTALL | re.IGNORECASE)
#     tech_skills = re.search(r"TECHNICAL_SKILLS:(.*)", result, re.DOTALL | re.IGNORECASE)

#     # Fill EY Template
#     with open(ey_template_path, "r", encoding="utf-8") as f:
#         template = f.read()

#     final_resume = template
#     final_resume = final_resume.replace("[CANDIDATE_NAME]", candidate_name)
#     final_resume = final_resume.replace("[ROLE]", prompt_text)
#     final_resume = final_resume.replace("[SUMMARY]", summary.group(1).strip() if summary else "")
#     final_resume = final_resume.replace("[QUALIFICATIONS]", qualifications.group(1).strip() if qualifications else "")
#     final_resume = final_resume.replace("[EXPERIENCE_PLACEHOLDER]", experience.group(1).strip() if experience else "")
#     final_resume = final_resume.replace("[TECHNICAL_SKILLS]", tech_skills.group(1).strip() if tech_skills else "")

#     # Save
#     output_path = os.path.join(output_dir, f"{candidate_name}_customised_resume_RAG.txt")
#     with open(output_path, "w", encoding="utf-8") as f:
#         f.write(final_resume)

#     print("\n✅ Resume customized and saved to:", output_path)

#     # Accuracy Metrics
#     tfidf_score = compute_tfidf_similarity(enriched_jd, final_resume)
#     keyword_match_ratio, matched_keywords = keyword_coverage(enriched_jd, final_resume)
#     qualitative_score = qualitative_relevance(enriched_jd, final_resume)
#     section_flags = section_audit(final_resume)

#     print("\n📊 Accuracy Metrics:")
#     print(f"TF-IDF Similarity:            {tfidf_score:.2f}")
#     print(f"Keyword Coverage Ratio:       {keyword_match_ratio:.2f}")
#     print(f"Qualitative Relevance:        {qualitative_score:.2f}")
#     print(f"Matched Keywords:             {matched_keywords}")
#     print(f"Section Presence Flags:       {section_flags}\n")

#     return output_path

# # ------------------- RUN EXAMPLE -------------------
# prompt = "Create a JD for a retail IRB model validation requirement. Mention that SQL is a must."
# resume_path = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\Rawresumes\Kundan_Kumar.pdf"

# customise_resume(prompt, resume_path)



Step 1: Retrieving closest-matching JD from local JD folder for → Create a JD for a retail IRB model validation requirement. Mention that SQL is a must.
✅ JD Retrieved From: C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\JDs\JD_11.txt

Step 2: Reading uploaded resume → C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\Rawresumes\Kundan_Kumar.pdf

Step 3: If resume sparse, enrich from closest historical resume

Step 4: Sending both JD + resume to LLM for customization
LLM call failed: HTTPConnectionPool(host='localhost', port=1234): Read timed out. (read timeout=180)
❌ Customization failed.


In [ ]:
import os
import fitz  # PyMuPDF
import docx
import re
import json
import requests
import time
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ---- CONFIGURATION ----
ey_template_path = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\EY_sample_resume_template.txt"
output_dir = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\final_customised_resumes"
jd_folder = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\JDs"
historical_resume_folder = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\Rawresumes"

os.makedirs(output_dir, exist_ok=True)
nlp = spacy.load("en_core_web_sm")


# ---- READERS ----
def read_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join([page.get_text() for page in doc])

def read_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])

def read_resume_text(file_path):
    if file_path.endswith(".pdf"):
        return read_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return read_text_from_docx(file_path)
    else:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()

def get_closest_match_from_folder(prompt, folder_path):
    max_score = 0
    best_file = None
    prompt_doc = nlp(prompt.lower())
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                text = read_resume_text(file_path)
                score = prompt_doc.similarity(nlp(text.lower()))
                if score > max_score:
                    max_score = score
                    best_file = file_path
            except Exception:
                continue
    return best_file


# ---- METRICS ----
def compute_tfidf_similarity(jd, resume):
    tfidf = TfidfVectorizer().fit_transform([jd, resume])
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

def keyword_coverage(jd, resume):
    jd_keywords = set(token.text.lower() for token in nlp(jd) if token.is_alpha and not token.is_stop)
    resume_words = resume.lower()
    matched = [kw for kw in jd_keywords if kw in resume_words]
    return len(matched) / len(jd_keywords) if jd_keywords else 0, matched

def qualitative_relevance(jd, resume):
    jd_words = set(jd.lower().split())
    resume_words = set(resume.lower().split())
    overlap = jd_words.intersection(resume_words)
    return len(overlap) / len(jd_words) if jd_words else 0

def section_audit(text):
    sections = ["summary", "qualification", "experience", "technical", "skill"]
    present = [sec for sec in sections if sec in text.lower()]
    return present


# ---- LLM STUDIO CALL ----
def call_lmstudio(prompt, model="zephyr-7b-beta.Q4_K_M.gguf", retries=2):
    url = "http://127.0.0.1:1234/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "stream": False
    }

    for attempt in range(retries):
        try:
            print(f"Sending prompt with {len(prompt)} characters to LLM...")
            response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=600)
            return response.json()['choices'][0]['message']['content']
        except Exception as e:
            print(f"LLM call failed: {e}")
            time.sleep(2)
    return ""


# ---- MAIN FUNCTION ----
def customise_resume(prompt_text, resume_path):
    candidate_name = os.path.splitext(os.path.basename(resume_path))[0]

    print(f"Step 1: Retrieving closest-matching JD from local JD folder for → {prompt_text}")
    matched_jd_file = get_closest_match_from_folder(prompt_text, jd_folder)
    if not matched_jd_file:
        print("❌ No JD file found in folder.")
        return

    with open(matched_jd_file, "r", encoding="utf-8") as f:
        jd_text = f.read()

    print("✅ JD Retrieved From:", matched_jd_file)

    print(f"\nStep 2: Reading uploaded resume → {resume_path}")
    resume_text = read_resume_text(resume_path)

    print("\nStep 3: If resume sparse, enrich from closest historical resume")
    hist_resume_file = get_closest_match_from_folder(prompt_text, historical_resume_folder)
    hist_resume_text = read_resume_text(hist_resume_file) if hist_resume_file else ""

    enriched_resume_text = resume_text + "\n\n" + hist_resume_text

    # Optional: truncate if too long
    if len(enriched_resume_text) > 18000:
        enriched_resume_text = enriched_resume_text[:18000]
        print("🔁 Resume text truncated for LLM compatibility.")

    print("\nStep 4: Sending both JD + resume to LLM for customization")
    llm_prompt = f"""
You are a resume writer. Given the candidate's resume and the job description, rewrite the following sections to align the resume with the JD. DO NOT INVENT DETAILS. Use only content present in the resume or similar past resumes.

Resume:
{enriched_resume_text}

Job Description:
{jd_text}

Output Format:
SUMMARY:
...

QUALIFICATIONS:
...

EXPERIENCE:
...

TECHNICAL_SKILLS:
...
"""

    result = call_lmstudio(llm_prompt)
    if not result:
        print("❌ Customization failed.")
        return

    print("\n=== Raw Output from LLM ===\n", result)

    summary = re.search(r"SUMMARY:(.*?)QUALIFICATIONS:", result, re.DOTALL | re.IGNORECASE)
    qualifications = re.search(r"QUALIFICATIONS:(.*?)EXPERIENCE:", result, re.DOTALL | re.IGNORECASE)
    experience = re.search(r"EXPERIENCE:(.*?)TECHNICAL_SKILLS:", result, re.DOTALL | re.IGNORECASE)
    tech_skills = re.search(r"TECHNICAL_SKILLS:(.*)", result, re.DOTALL | re.IGNORECASE)

    with open(ey_template_path, "r", encoding="utf-8") as f:
        template = f.read()

    final_resume = template
    final_resume = final_resume.replace("[CANDIDATE_NAME]", candidate_name)
    final_resume = final_resume.replace("[ROLE]", prompt_text)
    final_resume = final_resume.replace("[SUMMARY]", summary.group(1).strip() if summary else "")
    final_resume = final_resume.replace("[QUALIFICATIONS]", qualifications.group(1).strip() if qualifications else "")
    final_resume = final_resume.replace("[EXPERIENCE_PLACEHOLDER]", experience.group(1).strip() if experience else "")
    final_resume = final_resume.replace("[TECHNICAL_SKILLS]", tech_skills.group(1).strip() if tech_skills else "")

    output_path = os.path.join(output_dir, f"{candidate_name}_customised.txt")
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(final_resume)

    print("✅ Resume customized and saved to:", output_path)

    tfidf_score = compute_tfidf_similarity(jd_text, final_resume)
    keyword_match_ratio, matched_keywords = keyword_coverage(jd_text, final_resume)
    qualitative_score = qualitative_relevance(jd_text, final_resume)
    section_flags = section_audit(final_resume)

    print("\nAccuracy Metrics:")
    print(f"TF-IDF Similarity:            {tfidf_score:.2f}")
    print(f"Keyword Coverage Ratio:       {keyword_match_ratio:.2f}")
    print(f"Qualitative Relevance:        {qualitative_score:.2f}")
    print(f"Matched Keywords:             {matched_keywords}")
    print(f"Section Presence Flags:       {section_flags}\n")


# ---- RUN EXAMPLE ----
prompt = "Create a JD for a retail IRB model validation requirement. Mention that SQL is a must."
resume_path = r"C:\Users\HN338QQ\OneDrive - EY\Documents\raw and ey curated samples\Rawresumes\Kundan_Kumar.pdf"

customise_resume(prompt, resume_path)
